<a href="https://colab.research.google.com/github/michele1783/ADM-HW3/blob/main/parsing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive

In [2]:
from google.colab import files

In [84]:
import requests
from bs4 import BeautifulSoup
import shutil


import csv

In [87]:
def extractData(pagePath):
    with open(pagePath, encoding="utf-8") as fp:
        soup = BeautifulSoup(fp, "html.parser")
        animeTitle = " ".join((soup.html.head.title.text).split())
        
        #print("animeTitle: ", animeTitle)
        animeInfo = (soup.find_all("div",  attrs={ "class" : "spaceit_pad"}))
        #print("Animeinfo: ", animeInfo)
        
        animeType = " ".join((findField(animeInfo, "Type:").a.text).split())
        #print("animeType :", animeType)
        
        animeNumEpisode = int(list(filter(lambda y: y != '',list(map(lambda x: " ".join((x).split()),findField(animeInfo, "Episodes:").findAll(text=True, recursive=False)) )))[0])
        #print("animeNumEpisode: ", animeNumEpisode)
        
        aired = list(filter(lambda y: y != '',list(map(lambda x: " ".join((x).split()),findField(animeInfo, "Aired:").findAll(text=True, recursive=False)) )))[0].split('to')
        releaseDate = datetime.datetime(int(aired[0].split()[2]), months.index(aired[0].split()[0])+1 , int(aired[0].split()[1][:1]))
        if len(aired) == 2:
            endDate = datetime.datetime(int(aired[1].split()[2]), months.index(aired[1].split()[0])+1 , int(aired[1].split()[1][:1]))
        else:
            endDate = ""
        #print("releaseDate: ", releaseDate)
        #print("endDate: ", endDate)
        
        animeNumMembers = int(list(filter(lambda y: y != '',list(map(lambda x: " ".join((x).split()),findField(animeInfo, "Members:").findAll(text=True, recursive=False)) )))[0].replace(",", ""))
        #print("animeNumMembers: ", animeNumMembers)
        
        animeScore = float(" ".join((soup.find_all("span", itemprop = "ratingValue")[0].text).split()))
        #print("animeScore: ", animeScore)
        
        animeUsers = int(" ".join((soup.find_all("span", itemprop = "ratingCount")[0].text).split()))
        #print("animeUsers: ", animeUsers)
        
        animeRank = int(list(filter(lambda x: x[0] == '#',findField(animeInfo, "Ranked:").text.split()))[0].replace('#',''))
        #print("animeRank: ", animeRank)

        animePopularity = int(list(filter(lambda y: y != '',list(map(lambda x: " ".join((x).split()),findField(animeInfo, "Popularity:").findAll(text=True, recursive=False)) )))[0].replace("#", ""))
        #print("animePopularity: ", animePopularity)
        
        animeDescription = soup.find_all("p", itemprop = "description")[0].text.strip().replace('\n', '').replace('  ', '')
        #print("animeDescription: ", animeDescription)
        
        animeRelatedAHref = soup.find(name="table",attrs={"class":"anime_detail_related_anime"}).findChildren('a', href=True)
        animeRelated = []
        for x in animeRelatedAHref:
            aux = " ".join((x.text).split())
            if aux not in animeRelated:
                animeRelated.append(aux)
        #print("animeRelated: ", animeRelated)
        
        animeCharactersRaw = soup.find_all("h3", attrs={"class" : "h3_characters_voice_actors"})
        animeCharacters = []
        for x in animeCharactersRaw:
            animeCharacters.append(" ".join((x.text).split()))
        #print("animeCharacters: ", animeCharacters)
        
        animeVoicesRaw = soup.find_all("td", attrs={"class" : "va-t ar pl4 pr4"})
        animeVoices = []
        for x in animeVoicesRaw:
            animeVoices.append(" ".join((x.contents[1].text).split()))
        #print("animeVoices: ", animeVoices)
        
        animeStaff = ""
        if(len(soup.find_all("div", attrs={"class" : "detail-characters-list clearfix"})) == 2):
            aux = soup.find_all("div", attrs={"class" : "detail-characters-list clearfix"})[1]
            animeStaffRaw = []
            for x in aux.findChildren('a'):
                if (" ".join((x.text).split()) != ''):
                    animeStaffRaw.append(" ".join((x.text).split()))
            animeStaffTaskRaw = []
            for x in aux.findChildren('small'):
                animeStaffTaskRaw.append(" ".join((x.text).split()))
            #print("animeStaffRaw: ", animeStaffRaw)
            #print("animeStaffTaskRaw: ", animeStaffTaskRaw)
            animeStaff = [list(a) for a in zip(animeStaffRaw, animeStaffTaskRaw)]

        
        #print("animeStaff: ", animeStaff)
        
        return [animeTitle, animeType, animeNumEpisode, releaseDate, endDate, animeNumMembers, animeScore, animeUsers, animeRank, animePopularity, animeDescription, animeRelated, animeCharacters, animeVoices, animeStaff]

In [3]:
drive.mount('/content/drive')

Mounted at /content/drive


In [72]:
path = '/content/drive/MyDrive/ADM-HW3/DM'
urls = '/content/drive/MyDrive/ADM-HW3/urls.txt'

In [49]:
a = open(urls, "r")
enum = enumerate(a)
d=dict((i,j) for i,j in enum)

In [81]:
import os
k = 9950
for i in range(200,401):
    for j in range(1,51):
      k = k + 1
      try:
        file_size = os.path.getsize(path +  "/cartella{}/page_{}.html".format(i, k))
      except FileNotFoundError: 
          page = requests.get(d[k])
          soup = BeautifulSoup(page.content, features ="lxml")
          f = open("page_{}.html".format(k), "w")
          f.write(soup.prettify())
          shutil.move("page_{}.html".format(k), path + "/cartella{}".format(i))
          f.close()
      if file_size < 50000:
          os.remove(path +  "/cartella{}/page_{}.html".format(i, k))
          page = requests.get(d[k])
          soup = BeautifulSoup(page.content, features ="lxml")
          f = open("page_{}.html".format(k), "w")
          f.write(soup.prettify())
          shutil.move("page_{}.html".format(k), path + "/cartella{}".format(i))
          f.close()
          break




In [69]:
path = '/content/drive/MyDrive/ADM-HW3/nFolders400'

In [70]:
import os
k = 0
for i in range(1,200):
    for j in range(1,51):
      k = k + 1
      try:
        file_size = os.path.getsize(path +  "/cartella{}/page_{}.html".format(i, k))
      except FileNotFoundError: 
          page = requests.get(d[k])
          soup = BeautifulSoup(page.content, features ="lxml")
          f = open("page_{}.html".format(k), "w")
          f.write(soup.prettify())
          shutil.move("page_{}.html".format(k), path + "/cartella{}".format(i))
          f.close()
      if file_size < 50000:
          os.remove(path +  "/cartella{}/page_{}.html".format(i, k))
          page = requests.get(d[k])
          soup = BeautifulSoup(page.content, features ="lxml")
          f = open("page_{}.html".format(k), "w")
          f.write(soup.prettify())
          shutil.move("page_{}.html".format(k), path + "/cartella{}".format(i))
          f.close()
          break

In [103]:
path_1 = '/content/drive/MyDrive/ADM-HW3/DM'

In [98]:
def findField(array, word):
    for x in array:
        #print("#####################################")
        #print(" ".join((x.find("span").text).split()))
        #print("#####################################")
        if " ".join((x.find("span").text).split()) == word:
            return x
    return -1

In [102]:
import datetime

In [105]:
nFolders = 400
c = 9950
url_file = urls
num_lines = sum(1 for line in open(url_file))
pagesPerFolder = num_lines/nFolders
outputTSV = "dataset.tsv"

months = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun','Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
with open(outputTSV, 'wt') as out_file:
    tsv_writer = csv.writer(out_file, delimiter='\t')
    tsv_writer.writerow(["animeTitle", "animeType", "animeNumEpisode", "releaseDate", "endDate", "animeNumMembers", "animeScore", "animeUsers", "animeRank", "animePopularity", "animeDescription", "animeRelated", "animeCharacters", "animeVoices", "animeStaff"])
    for x in range(9951, 20000 +1):

        number = int(c/(pagesPerFolder)) + 1
        path = path_1 + "/cartella{}".format(number) + "/page_{}.html".format(c+1)
        print("path: ", path)
        tsv_writer.writerow(extractData(path))
        c = c + 1

path:  /content/drive/MyDrive/ADM-HW3/DM/cartella200/page_9951.html
path:  /content/drive/MyDrive/ADM-HW3/DM/cartella200/page_9952.html
path:  /content/drive/MyDrive/ADM-HW3/DM/cartella200/page_9953.html
path:  /content/drive/MyDrive/ADM-HW3/DM/cartella200/page_9954.html
path:  /content/drive/MyDrive/ADM-HW3/DM/cartella200/page_9955.html
path:  /content/drive/MyDrive/ADM-HW3/DM/cartella200/page_9956.html
path:  /content/drive/MyDrive/ADM-HW3/DM/cartella200/page_9957.html
path:  /content/drive/MyDrive/ADM-HW3/DM/cartella200/page_9958.html
path:  /content/drive/MyDrive/ADM-HW3/DM/cartella200/page_9959.html
path:  /content/drive/MyDrive/ADM-HW3/DM/cartella200/page_9960.html
path:  /content/drive/MyDrive/ADM-HW3/DM/cartella200/page_9961.html
path:  /content/drive/MyDrive/ADM-HW3/DM/cartella200/page_9962.html
path:  /content/drive/MyDrive/ADM-HW3/DM/cartella200/page_9963.html
path:  /content/drive/MyDrive/ADM-HW3/DM/cartella200/page_9964.html
path:  /content/drive/MyDrive/ADM-HW3/DM/cartell

AttributeError: ignored

In [94]:
nFolders = 400
c = 200
url_file = urls
num_lines = sum(1 for line in open(url_file))
pagesPerFolder = num_lines/nFolders
outputTSV = "dataset.tsv"
print(pagesPerFolder)

50.0
